<a href="https://colab.research.google.com/github/wyxuan02/GenerativeAI_class2025/blob/main/AI_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1:  安裝並引入必要套件

In [ ]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
import openai
import gradio as gr
import pandas as pd
import numpy as np

## Step 2:  TDEE 計算與營養分析

In [ ]:
def calculate_tdee_verbose(sex, age, weight, height, activity_level, goal):
    # 1. 計算 BMR（基礎代謝率）
    if sex == "男性":
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161

    # 2. 活動係數
    activity_factors = {
        "久坐（幾乎不運動）": 1.2,
        "輕度活動（每週1-3天）": 1.375,
        "中度活動（每週3-5天）": 1.55,
        "高強度活動（每週6-7天）": 1.725,
        "非常高（每日訓練、勞力工作）": 1.9,
    }
    activity_factor = activity_factors[activity_level]
    tdee = bmr * activity_factor

    # 3. 根據目標調整 TDEE
    if goal == "減脂":
        adjusted_tdee = tdee * 0.85
    elif goal == "增肌":
        adjusted_tdee = tdee * 1.15
    else:
        adjusted_tdee = tdee

    # 4. 計算營養素比例
    if goal == "增肌":
        ratio = {"蛋白質": 0.30, "碳水化合物": 0.45, "脂肪": 0.25}
    elif goal == "減脂":
        ratio = {"蛋白質": 0.35, "碳水化合物": 0.35, "脂肪": 0.30}
    else:
        ratio = {"蛋白質": 0.25, "碳水化合物": 0.50, "脂肪": 0.25}

    protein = (adjusted_tdee * ratio["蛋白質"]) / 4
    carbs = (adjusted_tdee * ratio["碳水化合物"]) / 4
    fat = (adjusted_tdee * ratio["脂肪"]) / 9

    # 格式化輸出
    output = f"""
🔍 你的健康分析結果：

📌 BMR（基礎代謝率）：{int(bmr)} kcal/day
🏃 TDEE（總每日能量消耗）：{int(tdee)} kcal
🎯 根據目標調整後建議攝取熱量：{int(adjusted_tdee)} kcal

📊 建議三大營養素攝取量：
🍗 蛋白質：{int(protein)} 克
🍚 碳水化合物：{int(carbs)} 克
🥑 脂肪：{int(fat)} 克
"""
    return output, {
        "protein": int(protein),
        "carbs": int(carbs),
        "fat": int(fat)
    }


## Step 3: 準備營養知識資料庫


> 手動整理知識資料庫，參考來源為 [衛生福利部國民健康署](https://www.hpa.gov.tw/Pages/List.aspx?nodeid=36)


> 餐食建議參考 ChatGPT 所整理的菜單來進行調整





In [ ]:
# prompt: 用 Linux 指令，讀入一個在網路上，網址是 URL 的檔案。

URL = "https://drive.google.com/uc?export=download&id=1wpbUpetvpyWxicTKArvQXkFvShT6ewry"

!wget -O healthymeal_faiss_db.zip "$URL"


--2025-05-31 08:46:43--  https://drive.google.com/uc?export=download&id=1wpbUpetvpyWxicTKArvQXkFvShT6ewry
Resolving drive.google.com (drive.google.com)... 64.233.179.113, 64.233.179.102, 64.233.179.101, ...
Connecting to drive.google.com (drive.google.com)|64.233.179.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1wpbUpetvpyWxicTKArvQXkFvShT6ewry&export=download [following]
--2025-05-31 08:46:43--  https://drive.usercontent.google.com/download?id=1wpbUpetvpyWxicTKArvQXkFvShT6ewry&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 209.85.145.132, 2607:f8b0:4001:c1e::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|209.85.145.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38242 (37K) [application/octet-stream]
Saving to: ‘healthymeal_faiss_db.zip’

healthymeal_faiss_d 100%[===================>]  37.35K  -

In [ ]:
!unzip healthymeal_faiss_db.zip

Archive:  healthymeal_faiss_db.zip
replace healthymeal_faiss_db/index.faiss? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: healthymeal_faiss_db/index.faiss  
replace healthymeal_faiss_db/index.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: healthymeal_faiss_db/index.pkl  


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("healthymeal_faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

## Step 4: 建立 LLM



In [ ]:
import os
from google.colab import userdata

api_key = userdata.get('Groq')
os.environ["OPENAI_API_KEY"] = api_key

model = "llama3-70b-8192"
base_url="https://api.groq.com/openai/v1"

client = OpenAI(
    base_url=base_url # 使用 OpenAI 本身不需要這段
)

In [ ]:
system_prompt = '''
你是一位專業且知名的健康私廚，同時熟悉營養學與六大類食物原則。你的任務是根據使用者的健康目標與建議營養攝取（蛋白質、碳水化合物、脂肪）推薦具體的健康餐點組合。

你的推薦須符合以下原則：
1. 餐點必須符合使用者的營養建議攝取量（允許誤差 ±10%）。
2. 若有飲食風格（如：素食、生酮、地中海）、過敏或疾病限制，請一併考量。
3. 請使用六大類食物與知識庫資料，組合出合理菜單（含食材、製作方式、營養含量）。
4. 若有需要，請提供替代食材（例如：乳糖不耐 → 換豆漿），並註明總熱量與 GI 值。

請用台灣習慣的繁體中文回應，並避免提供任何非飲食相關建議。
如果是其他語言，請翻譯成繁體中文。
'''

prompt_template = """
你已取得以下資料摘要供參考（包括食材特性、營養建議、疾病飲食注意事項與常見餐點組合）：

{retrieved_chunks}

使用者的問題是：{question}

請綜合資料內容與使用者提供的營養建議值，推薦一份「符合健康目標的具體餐點」：
- 餐點應包含食材名稱、製作方式、三大營養素含量（蛋白質、碳水、脂肪）、總熱量（kcal）、GI 值（如資料有）
- 若食材有過敏或限制情況，請主動提出替代方案
- 餐點建議以實際操作可行為主，避免過於理論或模糊描述
- 若資料不足請明確說明「目前資料不足，敬請期待更新菜單」

請用繁體中文回答。
如果是其他語言，請翻譯成繁體中文。
"""


In [ ]:
chat_history = []

def chat_with_rag(user_input, nutrient_targets=None):

    # RAG：檢索食材特性與健康建議
    docs = retriever.get_relevant_documents(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    # 如果提供營養目標值 → 融入 prompt 中產生個性化菜單
    if nutrient_targets:
        user_input += f"\n請依據以下建議值推薦菜單：蛋白質{nutrient_targets['protein']}g、碳水{nutrient_targets['carbs']}g、脂肪{nutrient_targets['fat']}g"


    final_prompt = prompt_template.format(retrieved_chunks=retrieved_chunks, question=user_input)

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": final_prompt},
        ]
    )
    return response.choices[0].message.content


In [ ]:
# 自訂 CSS

custom_css = """

/* === 整體背景與版面 === */
body, html {
    background-color: #f6f9fc !important;
    margin: 0;
    padding: 0;
    font-family: "Segoe UI", "Helvetica Neue", sans-serif;
}

.gradio-container {
    background-color: #f6f9fc !important;
    box-shadow: none !important;
}

/* === 標題與段落 === */
h1 {
    font-size: 36px;
    font-weight: bold;
    color: #1a2b4c;
    text-align: center;
    margin-top: 20px;
}

p {
    font-size: 18px;
    color: #444;
    line-height: 1.6;
    text-align: left;
    margin: 10px 0;
}

/* === 文字輸入與區塊 === */
input, textarea, .gr-textbox {
    background-color: #ffffff;
    color: #333333;
    border: 1px solid #ccd6dd;
    border-radius: 8px;
    padding: 10px;
    font-size: 16px;
}

/* === 按鈕樣式 === */
button {
    background-color: #1a73e8;
    color: white;
    font-weight: bold;
    border-radius: 6px;
    padding: 10px 20px;
    border: none;
    transition: background-color 0.2s ease-in-out;
}

button:hover {
    background-color: #0c56d0;
}

/* === Chatbot 對話氣泡 === */
.gr-chatbot .message {
    border-radius: 8px;
    padding: 12px;
    margin: 6px 0;
    white-space: pre-wrap;
    text-align: left;
    font-size: 16px;
    line-height: 1.6;
}

.gr-chatbot .message.user {
    background-color: #e3f2fd !important;
    color: #1a2b4c !important;
    border-left: 4px solid #1a73e8;
}

.gr-chatbot .message.assistant {
    background-color: #ffffff !important;
    color: #333333 !important;
    border-left: 4px solid #4caf50;
}

"""

In [ ]:
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("# 🥗 你的專屬健康飲食顧問 👨‍🍳️💡")

    with gr.Row():
        with gr.Column(scale=1):
            sex = gr.Dropdown(["男性", "女性"], label="性別")
            age = gr.Slider(10, 80, value=25, label="年齡")
            weight = gr.Slider(30, 150, value=60, label="體重（kg）")
            height = gr.Slider(130, 210, value=170, label="身高（cm）")
            activity = gr.Dropdown([
                "久坐（幾乎不運動）", "輕度活動（每週1-3天）", "中度活動（每週3-5天）",
                "高強度活動（每週6-7天）", "非常高（每日訓練、勞力工作）"], label="活動量")
            goal = gr.Dropdown(["減脂", "增肌", "維持", "控糖"], label="健康目標")
            btn = gr.Button("📈 計算熱量與營養建議")

        with gr.Column(scale=2):
            tdee_output = gr.Textbox(lines=8, label="📋 TDEE 與營養建議")
            recommend_output = gr.Textbox(lines=18, label="🥗 三餐健康餐點推薦")

    def recommend_from_tdee(sex, age, weight, height, activity, goal):
        try:
            result_text, nutrient_targets = calculate_tdee_verbose(
                sex, age, weight, height, activity, goal
            )
            breakfast = chat_with_rag("請依據上述營養素建議提供一份健康早餐", nutrient_targets)
            lunch = chat_with_rag("請依據上述營養素建議提供一份健康中餐", nutrient_targets)
            dinner = chat_with_rag("請依據上述營養素建議提供一份健康晚餐", nutrient_targets)

            full_recommendation = f"""\n🍽 **健康三餐建議**\n\n🥣 **早餐**：\n{breakfast}\n\n🍱 **午餐**：\n{lunch}\n\n🍛 **晚餐**：\n{dinner}"""
            return result_text, full_recommendation
        except Exception as e:
            return f"⚠️ 錯誤：{e}", "❌ 無法產生推薦餐點，請稍後再試或檢查輸入。"

    btn.click(
        fn=recommend_from_tdee,
        inputs=[sex, age, weight, height, activity, goal],
        outputs=[tdee_output, recommend_output]
    )

    gr.Markdown("---")
    gr.Markdown("## 🤖 營養助理互動區（可問任何問題）")

    chatbot = gr.Chatbot(label="飲食助理", type="messages")
    msg = gr.Textbox(placeholder="請輸入你的飲食問題...", show_label=False)

    def respond(message, chat_history_local):
        try:
            response = chat_with_rag(message)
            chat_history_local.append({"role": "user", "content": message})
            chat_history_local.append({"role": "assistant", "content": response})
            return "", chat_history_local
        except Exception as e:
            error_msg = f"❌ 錯誤：{str(e)}"
            chat_history_local.append({"role": "user", "content": message})
            chat_history_local.append({"role": "assistant", "content": error_msg})
            return "", chat_history_local

    msg.submit(respond, [msg, chatbot], [msg, chatbot])


In [ ]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36453cbd54e86efe60.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
